In [1]:
suppressMessages(suppressWarnings(library(tidyverse)))

In [2]:
cs2go <- read.table("../../ANALYSIS/cS2G/ALL_CS2G.TAB",header=FALSE)
colnames(cs2go) <- c("MARKERNAME","GENENAME","SCORE","INFO")

In [3]:
### TRANS

In [4]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/TRANSANCESTRY_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_THREE <- read.table("../data/TRANSANCESTRY_GWAS_ALLELECORRECTED.TAB",header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_AGEN","PVALUE_METSIM","NA1","NA2","NA3")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
TRANSANCESTRY_RESULTS <- merge(META_THREE,Metasoft_results,by="MARKERNAME")
TRANSANCESTRY_RESULTS <- TRANSANCESTRY_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(AGEN_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+AGEN_EAF*2*AGEN_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(AGEN_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2,-NA3)

In [5]:
head(TRANSANCESTRY_RESULTS,1)

,MARKERNAME,CHROMOSOME,POSITION,ADIPO_EA,ADIPO_NEA,ADIPO_EAF,ADIPO_N,ADIPO_BETA,ADIPO_SE,ADIPO_PVALUE,...,PVALUE_BE,I_SQUARE,Q,PVALUE_Q,TAU_SQUARE,PVALUE_ADIPOGEN,PVALUE_AGEN,PVALUE_METSIM,TOTAL_N,MAF_AVG
,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,...,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,rs10000081,4,17348363,T,C,0.2458,29318,-0.011588,0.005128,0.0284451,...,NA,0,0.893958,0.639557,0,0.0238367,0.989502,0.96363,46381,0.3322903


In [ ]:
## OVERLAP

In [7]:
trans_cs2go <- merge(cs2go,TRANSANCESTRY_RESULTS,by="MARKERNAME")%>%filter(PVALUE_FE < 0.05)%>%select(CHROMOSOME,POSITION,GENENAME,SCORE,INFO)

In [9]:
write.table(trans_cs2go,"../../ANALYSIS/cS2G/TRANS_CS2G.TAB",row.names=FALSE,quote=FALSE,sep="\t")

In [10]:
## EUR

In [11]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/EUROPEAN_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_TWO <- read.table(gzfile("../data/EUROPEAN_GWAS_ALLELECORRECTED.TAB"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_METSIM","NA1","NA2")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
EUR_RESULTS <- merge(META_TWO,Metasoft_results,by="MARKERNAME")
EUR_RESULTS <- EUR_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2)

In [ ]:
#OVERLAP

In [12]:
eur_cs2go <- merge(cs2go,EUR_RESULTS,by="MARKERNAME")%>%filter(PVALUE_FE < 0.05)%>%select(CHROMOSOME,POSITION,GENENAME,SCORE,INFO)

In [13]:
write.table(eur_cs2go,"../../ANALYSIS/cS2G/EUR_CS2G.TAB",row.names=FALSE,quote=FALSE,sep="\t")